In [1]:
# 1. scrape today's games with tipoff time (@ like 12pm)
# 2. create a custom yml to run 30 minutes before each of the games on this day
# 3. the script -- for all relevant games -- calls the odds api to fetch and save odds

In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import os
import re
import calendar
from time import sleep

from helpers.scrape import get_first_basket

from helpers.utils import getId

In [1]:
from helpers.preprocess import feature_engineering
data, features = feature_engineering()

In [23]:
games = data.copy()[data['game_id'] == '202412090TOR'].reset_index(drop = True)

X = games[features].to_numpy()
games[['game_id', 'player_id', 'Player'] + features]

,game_id,player_id,Player,PTS_avg,PTS_5,PTS_25,PTS_50,USG%_avg,USG%_5,USG%_25,...,VORP_50,FGA_avg,FGA_5,FGA_25,FGA_50,first_basket_avg,first_basket_5,first_basket_25,first_basket_50,rating
0,202412090TOR,bridgmi01,Mikal Bridges,0.084761,0.193116,-0.550697,-0.297635,-0.441867,-0.172517,-0.518985,...,-0.756559,0.038467,0.530520,0.049569,0.135670,-0.191068,0.474336,0.802717,1.176662,0.245618
1,202412090TOR,hartjo01,Josh Hart,-0.799489,-0.791398,-0.932536,-0.761304,-1.093262,-1.238156,-1.305579,...,0.650609,-0.851993,-1.078924,-1.391395,-0.874319,-1.113895,0.474336,-0.656769,-0.588331,-0.105265
2,202412090TOR,brunsja01,Jalen Brunson,0.680087,1.177630,1.638515,1.829789,0.969751,1.055568,1.266345,...,1.601634,0.612576,0.470911,1.248451,1.808936,0.063585,-1.897344,-0.656769,-1.176662,1.824592
3,202412090TOR,townska01,Karl-Anthony Towns,1.813117,0.609641,1.027572,0.974046,1.537132,0.935359,0.939822,...,1.001191,1.432174,0.709348,0.649010,0.633127,1.977111,0.474336,0.072974,0.588331,0.947384
4,202412090TOR,anunoog01,OG Anunoby,-0.338755,-0.639934,-0.236740,-0.287407,-0.503009,-0.328464,-0.483841,...,-0.139652,-0.303738,-0.065570,-0.180985,-0.326613,-0.415060,-1.897344,-1.386512,-0.588331,0.245618
5,202412090TOR,dickgr01,Gradey Dick,-0.615242,-0.299141,-0.219770,-0.546516,-0.215647,0.074400,-0.060832,...,-1.486775,-0.473523,-0.005961,0.014986,-0.412035,-0.406382,0.474336,1.532460,0.000000,-1.157914
6,202412090TOR,barnesc01,Scottie Barnes,0.584615,0.798971,0.229952,0.421734,0.454992,0.925612,0.686144,...,0.786193,0.823740,1.007393,0.602899,0.618053,1.055137,0.474336,0.802717,1.176662,0.421060
7,202412090TOR,agbajoc01,Ochai Agbaji,-1.309491,-1.889510,-1.501053,-1.494310,-1.191601,-1.657265,-1.402705,...,-1.220450,-1.234053,-1.794232,-1.529727,-1.492372,-1.268319,0.474336,-1.386512,-1.764993,-1.684239
8,202412090TOR,poeltja01,Jakob Poeltl,-1.061962,-0.450605,-0.601609,-0.703345,-0.821962,-0.705337,-0.419303,...,-0.262646,-1.311666,-1.019315,-0.780426,-0.954716,-0.444412,0.474336,0.072974,0.588331,-0.456148
9,202412090TOR,barrerj01,RJ Barrett,0.962360,1.291228,1.146366,0.864948,1.305474,1.110800,1.298933,...,-0.173548,1.268016,1.245829,1.317617,0.864269,0.743303,0.474336,0.802717,0.588331,-0.280707


In [24]:
import pickle
# Load the model from the .pkl file
with open('models/model_rf.pkl', "rb") as f:
    loaded_model = pickle.load(f)

In [25]:
import pandas as pd

In [31]:
games = games[['game_id', 'player_id', 'Player']]
y_pred = loaded_model.predict_proba(X)[:, -1]
y_pred /= y_pred.sum()
games['Pred. prob (%)'] = y_pred
# games['Pred. odds'] = pd.Series(y_pred).apply(lambda x: round(1/x, 1))
games.groupby('game_id')['Pred. prob (%)'].transform('sum')

0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
5    1.0
6    1.0
7    1.0
8    1.0
9    1.0
Name: Pred. prob (%), dtype: float64

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
from time import sleep

from helpers.utils import getId


yst = datetime.now(ZoneInfo('America/New_York')) - timedelta(days = 1)


url = f'https://www.basketball-reference.com/boxscores/?month={yst.month}&day={yst.day}&year={yst.year}'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'lxml')

game_ids = [getId(x) for x in soup.find_all('a', href = True) if 'boxscores/pbp' in x['href']]

In [32]:
# Load data
years = [2019, 2020, 2021, 2022, 2023, 2024, 2025]
first_basket = pd.concat([pd.read_csv(f'data/first_basket_{year}.csv') for year in years])
dfs = []
for year in years :
    df = pd.read_csv(f'data/rosters.nosync/rosters_{year}.csv')
    df.insert(1, 'Year', year)
    dfs.append(df)
stats = pd.concat(dfs).reset_index(drop = True)
stats = pd.merge(stats, first_basket[['game_id', 'first_basket']], on = 'game_id', how = 'inner')
stats['first_basket_scorer'] = (stats['player_id'] == stats['first_basket']).astype(int)

In [33]:
features = []
for f in ['PTS', 'USG%', 'VORP', 'FGA'] :

    stats[f'{f}_avg'] = stats.groupby('player_id')[f'{f}'].expanding().mean().reset_index(level = 0, drop = True)
    stats[f'{f}_avg'] = stats.groupby('player_id')[f'{f}_avg'].shift()
    features.append(f'{f}_avg')

    for w in [25, 50] :
        stats[f'{f}_{w}'] = stats.groupby('player_id')[f'{f}'].rolling(w, min_periods = 1).mean().reset_index(level = 0, drop = True)
        stats[f'{f}_{w}'] = stats.groupby('player_id')[f'{f}_{w}'].shift()
        features.append(f'{f}_{w}')

In [34]:
starters = stats.copy()[stats['starter']]

In [35]:
starters['first_basket_avg'] = starters.groupby('player_id')['first_basket_scorer'].expanding().mean().reset_index(level = 0, drop = True)
starters['first_basket_avg'] = starters.groupby('player_id')['first_basket_avg'].shift()
features.append(f'first_basket_avg')

for w in [25, 50] :
    starters[f'first_basket_{w}'] = starters.groupby('player_id')['first_basket_scorer'].rolling(w, min_periods = 1).mean().reset_index(level = 0, drop = True)
    starters[f'first_basket_{w}'] = starters.groupby('player_id')[f'first_basket_{w}'].shift()
    features.append(f'first_basket_{w}')

In [36]:
ratings = pd.read_csv('data/player_metadata.csv')[['player_id', 'rating']]#.rename(columns = {'rating': 'Rating'})
ratings['rating'] = (
    (ratings['rating'] - ratings['rating'].mean())
    / ratings['rating'].std()
)

In [37]:
starters = starters.merge(
    ratings[['player_id', 'rating']],
    on = ['player_id'],
    how = 'left'
)
features.append('rating')

In [39]:
starters[starters['game_id'].isin(game_ids)]

,game_id,Year,Player,player_id,Team,MP,FG,FGA,FG%,3P,...,VORP_avg,VORP_25,VORP_50,FGA_avg,FGA_25,FGA_50,first_basket_avg,first_basket_25,first_basket_50,rating
74051,202412080ATL,2025,Trae Young,youngtr01,ATL,35.133333,6,13,0.462,2,...,3.383721,0.788000,1.586000,18.479070,16.320000,16.880000,0.088372,0.080000,0.040000,2.101835
74052,202412080ATL,2025,Dyson Daniels,daniedy01,ATL,33.516667,7,15,0.467,2,...,0.704930,1.220000,1.538000,5.605634,11.720000,8.100000,0.061224,0.080000,0.061224,-0.307521
74053,202412080ATL,2025,Zaccharie Risacher,risacza01,ATL,29.466667,3,11,0.273,0,...,-1.037500,-1.037500,-1.037500,10.416667,10.416667,10.416667,0.181818,0.181818,0.181818,-0.146897
74054,202412080ATL,2025,Larry Nance Jr.,nancela02,ATL,24.400000,6,11,0.545,2,...,1.797345,2.312000,2.150000,5.952802,4.760000,4.240000,0.062500,0.080000,0.100000,0.013727
74055,202412080ATL,2025,Clint Capela,capelca01,ATL,19.016667,4,7,0.571,0,...,2.300494,1.268000,1.048000,9.283951,7.800000,8.200000,0.109453,0.080000,0.120000,0.656221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74166,202412080WAS,2025,Jaylen Wells,wellsja01,MEM,29.783333,0,3,0.000,0,...,0.045833,0.045833,0.045833,9.375000,9.375000,9.375000,0.105263,0.105263,0.105263,-1.271263
74167,202412080WAS,2025,Brandon Clarke,clarkbr01,MEM,23.183333,6,10,0.600,0,...,2.012030,0.756000,1.594000,7.342105,5.800000,6.220000,0.090909,0.120000,0.090909,0.174350
74168,202412080WAS,2025,Scotty Pippen Jr.,pippesc02,MEM,22.316667,6,11,0.545,0,...,1.070588,0.856000,1.078000,8.274510,8.720000,8.440000,0.080000,0.080000,0.080000,-0.307521
74169,202412080WAS,2025,Marcus Smart,smartma01,MEM,20.966667,5,8,0.625,3,...,1.372034,0.708000,0.090000,9.689266,9.880000,10.420000,0.082781,0.120000,0.100000,0.656221


In [7]:
season = 2018
url = f'https://www.basketball-reference.com/leagues/NBA_{season}_games.html'
page = requests.get(url)
if page.status_code == 429 :
    raise ValueError('Rate limited...')
soup = BeautifulSoup(page.content, 'lxml')
month_urls = [x['href'] for x in soup.find_all('a', href = True) if 'games' in x['href'] 
                and any(m.lower() in x['href'] for m in months)]

for m, month_url in enumerate(month_urls) :

    games_monthly = get_monthly_games(month_url)
    games_monthly['season'] = season

    break

        # first_basket_info = []
        # for i, gameId in enumerate(games_monthly['game_id'])  :

        #     print(f'[{round(100*(i+1)/len(games_monthly))}%...] season :  {season-1}-{season}, month :  {month_url.split("-")[-1].split(".")[0]} ({gameId})')
        #     sleep(5)
        #     first_basket_info.append(get_first_basket(gameId))


        # first_basket_df = pd.concat(first_basket_info)
        # first_basket_df = games_monthly.merge(first_basket_df, on = ['game_id', 'Home', 'Away'], how = 'inner')
        # if m == 0 :
        #     first_basket_df.to_csv(f'data/first_basket_{season}.csv', index = False)
        # else :
        #     first_basket_df.to_csv(f'data/first_basket_{season}.csv', mode = 'a', header = False, index = False)


SSLError: HTTPSConnectionPool(host='www.basketball-reference.com', port=443): Max retries exceeded with url: /leagues/NBA_2018_games.html (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1147)')))

In [4]:
url = 'https://www.rotowire.com/basketball/nba-lineups.php'

In [5]:
page = requests.get(url)

In [19]:
url = "https://www.rotowire.com/basketball/nba-lineups.php"
response = requests.get(url)
html = response.text.replace("<!--", "").replace(
    "-->", ""
)  # uncomment tables
soup = BeautifulSoup(html, "lxml")

In [20]:
len([x for x in soup.find_all('table')])

1

In [23]:
pd.read_html(str(table))[0]

,0,1,2
0,📱NBA Sportsbook,🎁 NBA Bonus,🤑 NBA Promo Code
1,📱BetMGM,"🎁 $1,500 First Bet Offer",🤑 ROTOBONUS
2,📱Caesars Sportsbook,"🎁 $1,000 First Bet on Caesars",🤑 ROTO1000
3,📱Fanatics Sportsbook,"🎁 Bet & Get $1,000 in No Sweat Bets",🤑 CLICK HERE
4,📱DraftKings,"🎁 Bet $5, Win $150 In Bonus Bets",🤑 CLICK HERE
5,📱FanDuel,"🎁 Bet $5, Win $150 in Bonus Bets + 3 Months of...",🤑 CLICK HERE
6,📱bet365,"🎁 $1,000 First Bet Safety Net or Bet $5, Get $150",🤑 ROTOWIRE


In [24]:
soup.find_all('button', class_ = 'see-proj-minutes')

[<button class="see-proj-minutes" data-home="0" data-nickname="Bulls" data-team="CHI">Projected Minutes</button>,
 <button class="see-proj-minutes" data-home="1" data-nickname="Pistons" data-team="DET">Projected Minutes</button>,
 <button class="see-proj-minutes" data-home="0" data-nickname="76ers" data-team="PHI">Projected Minutes</button>,
 <button class="see-proj-minutes" data-home="1" data-nickname="Heat" data-team="MIA">Projected Minutes</button>,
 <button class="see-proj-minutes" data-home="0" data-nickname="Wizards" data-team="WAS">Projected Minutes</button>,
 <button class="see-proj-minutes" data-home="1" data-nickname="Knicks" data-team="NYK">Projected Minutes</button>,
 <button class="see-proj-minutes" data-home="0" data-nickname="Pacers" data-team="IND">Projected Minutes</button>,
 <button class="see-proj-minutes" data-home="1" data-nickname="Raptors" data-team="TOR">Projected Minutes</button>,
 <button class="see-proj-minutes" data-home="0" data-nickname="Rockets" data-team

In [44]:
lineup_away = soup.find('ul', class_ = 'lineup__list is-visit')
starting_lineup_tags = lineup_away.find_all('li')[1:6]
[x.text.split('\n')[-2] for x in starting_lineup_tags]

['Josh Giddey', 'Coby White', 'Zach LaVine', 'P. Williams', 'N. Vucevic']

In [47]:
lineup_home = soup.find('ul', class_ = 'lineup__list is-home')
starting_lineup_tags = lineup_home.find_all('li')[1:6]
[x.text.split('\n')[-2] for x in starting_lineup_tags]

['C. Cunningham', 'Jaden Ivey', 'T. Hardaway', 'Tobias Harris', 'Jalen Duren']

In [48]:
[x for x in starting_lineup_tags.find_all('a', href = True)]

AttributeError: 'list' object has no attribute 'find_all'

In [50]:
[x['href'] for x in lineup_home.find_all('a', href = True)]

['/basketball/player/cade-cunningham-5336',
 '/basketball/player/jaden-ivey-5696',
 '/basketball/player/tim-hardaway-3459',
 '/basketball/player/tobias-harris-3211',
 '/basketball/player/jalen-duren-5703',
 '/basketball/player/tim-hardaway-3459',
 '/basketball/player/jaden-ivey-5696',
 '/basketball/player/bobi-klintman-5976',
 '/basketball/player/ausar-thompson-5932']

In [1]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from pretty_html_table import build_table
import numpy as np
import pandas as pd

In [2]:
df = pd.DataFrame(np.array([[0,1], [2, 3]]), columns = ['col1', 'col2'])

In [3]:
# Specify the email contents
mail = MIMEMultipart()
html = """\
<html><head></head><body>{0}</body></html>
""".format(build_table(df, 'grey_light', text_align = 'right', font_family = 'arial', width_dict = ['100','200','200','100','100','100','100'], font_size = 10))
mail.attach(MIMEText(html, 'html'))


In [4]:

# Set my email address and the password key
my_mail  = 'martinbog19@gmail.com'
# with open('gmail_key.txt') as f:
#     password = f.read()

# Set the subject of the email
mail['Subject'] = 'test_df'


In [ ]:

# Send the email (to myself)
server.sendmail(my_mail, my_mail, mail.as_string())
q = server.quit()

Email sent successfully!


In [9]:
import os

def is_running_in_github_actions():
    return os.getenv("GITHUB_ACTIONS") == "true"

In [12]:
os.getenv("k")

In [13]:
import pandas as pd
from datetime import datetime, timezone, timedelta
from zoneinfo import ZoneInfo
import requests
import os
import json



if os.getenv("GITHUB_ACTIONS") == "true" :
  api_key = os.getenv('ODDS_API_KEY')
else :
  with open('secrets/odds_api_key.txt') as f:
    api_key = f.read()

with open('utils/odds_tm_map.json', 'r') as f :
  odds_tm_map = json.load(f)

games = pd.read_csv('data/games.csv')

# Store subset of games in the next 30 minutes
now = datetime.now(ZoneInfo('America/New_York'))
# games_now = games[(games['Time'] > now.strftime('%Y-%m-%d %H:%M:%S')) & (games['Time'] <= (now + timedelta(minutes = 30)).strftime('%Y-%m-%d %H:%M:%S'))]


games_now = games.tail(1)

# Iterate through games
for _, game in games_now.iterrows() :

    eventId = game['event_id']

    odds_response = requests.get(f'https://api.the-odds-api.com/v4/sports/basketball_nba/events/{eventId}/odds',
                             params = {'apiKey': api_key,
                                       'regions': 'us',
                                       'markets': 'player_first_basket',
                                       'oddsFormat': 'decimal'})
    

    bm_dfs = [pd.DataFrame(columns = ['name', 'price', 'bookmaker', 'update_time'])]
    for bookmaker in odds_response.json()['bookmakers'] :
        
        bm_df = pd.DataFrame(bookmaker['markets'][0]['outcomes'])
        bm_df['bookmaker'] = bookmaker['key']
        bm_df['update_time'] = bookmaker['markets'][0]['last_update']

        bm_dfs.append(
            bm_df
            .drop(columns = 'name')
            .rename(columns = {'description': 'name'})
        )

    game_df = pd.concat(bm_dfs).reset_index(drop = True)
    game_df['game_id'] = game['game_id']
    game_df['event_id'] = game['event_id']

    game_df['insert_timestamp_utc'] = datetime.now(timezone.utc)
    game_df.to_csv('data/odds_first_basket.csv', index = None, header = None, mode = 'a')

In [15]:
odds = pd.read_csv('data/odds_first_basket.csv')

In [21]:
odds[odds['game_id'] == '202412050GSW'].groupby('name').agg({'price': ['mean', 'min', 'max']})

price            
                         mean   min   max
name                                     
Alperen Sengun       6.375000   6.0   6.5
Andrew Wiggins       6.625000   6.5   7.0
Brandin Podziemski  10.875000   9.5  12.0
Buddy Hield          8.000000   8.0   8.0
Dillon Brooks        9.375000   9.0  10.5
Fred VanVleet        9.875000   9.5  10.0
Jabari Smith Jr     10.000000   9.5  11.0
Jalen Green          7.625000   7.0   8.0
Jonathan Kuminga     7.500000   6.5   8.5
Kevon Looney         8.625000   8.0  10.5
Moses Moody         12.333333  12.0  13.0